In [1]:
"""
@author: BOURGEAIS & MONCHICOURT
"""
#Pour la création de fenêtres, de canvas etc..
try:  # import as appropriate for 2.x vs. 3.x
    import tkinter as tk
    import tkinter.messagebox as tkMessageBox
    from datetime import datetime #Pour permettre de chronometrer le temps de jeu
except:
    import Tkinter as tk
    import tkMessageBox
    import array as arr
import json #Pour la sauvegarde des scores

In [2]:
class Defender(object):
    def __init__(self):
        #Position de mon vaisseau
        self.x =340
        self.y = 670
        #ID de mon vaisseau
        self.id = None
        #Balles tirées
        self.max_fired_bullets = 8 #Nombre max a tirer
        self.fired_bullets = [] #Chaque ID d'une balle en stockée dans une case de ce tableau
        self.img = tk.PhotoImage(file='vaisseauspatial.png') #Chargement de la photo que j'initialiserai dans le install_in
    def install_in(self,canvas):
        canvas.pack()
        self.id=canvas.create_image(self.x,self.y, image=self.img)#J'affiche l'image
    def move_in(self,canvas,dx):
        x = int(canvas.coords(self.id)[0])#Grace aux coordonnées recuperer je pourrais empecher mon vaisseau de sortir de la fenetre
        if x+dx<40:
            dx=0
        elif x+dx>int(canvas['width'])-30:
            dx=0
        else:
            canvas.move(self.id,dx,0)
    def fire(self, canvas):
        if len(self.fired_bullets)<self.max_fired_bullets:
            parite=len(self.fired_bullets)%2 #je crée une alternance entre chaque execution de manière à pouvoir tirer depuis chacun des ailes de mon vaisseau
            aBullet=Bullet(self)
            aBullet.install_in(canvas,parite)
            self.fired_bullets.append(aBullet)     

In [3]:
class Bullet(object):
    def __init__(self, shooter):
        self.speed = 50
        self.id = None
        self.shooter = shooter
        self.img = tk.PhotoImage(file='tiralie.png')
        self.radius=20
        self.alive=True  #N'ayant pas reussi a delete mon bullet sans engendrer de gros problèmes dans mon code j'ai decidé de le rendre inefficace apres qu'il ai touché un adversaire avec alive
    def install_in(self, canvas ,parite):
        x1 = canvas.coords(self.shooter.id)[0]
        y1 = canvas.coords(self.shooter.id)[1]
        y1=y1-50
        if (parite==0) : #je me sers de la parite pour savoir depuis quelle aile je dois tirer 
            self.id=canvas.create_image((x1+20),(y1+self.radius), image=self.img)
        elif (parite==1):
            self.id=canvas.create_image((x1-20),(y1+self.radius), image=self.img)
    def move_in(self, canvas):
        x1 = canvas.coords(self.id)[0]
        y1 = canvas.coords(self.id)[1]
        canvas.move(self.id,0,-self.speed) 
        if self.alive==False : #N'ayant pas reussi a delete mon bullet sans engendrer de gros problèmes dans mon code j'ai decidé de le masqué avec une image transparente
                self.img =tk.PhotoImage(file='missilemort.png')
                self.id=canvas.create_image(x1,y1, image=self.img)
                canvas.move(self.id,0,-self.speed)

In [4]:
class Alien(object):
    def __init__(self):
        self.x =50
        self.y = 50
        self.id = None
        self.alive = True
        self.speed=20
        self.img = tk.PhotoImage(file='mechants2.png')
    def touched_by(self,canvas,projectile):
            x1=canvas.coords(projectile.id)[0]
            y1=canvas.coords(projectile.id)[1]
            x2,y2=canvas.coords(self.id)
            if self.alive==True : #je regarde si l'alien est vivant puisque je n'ai pas non plus reussi a le delete ça me permettra deviter que je tire en boucle sur le même alien en premiere ligne
                if (x2-40<x1<x2+40) and (y2-50<y1<y2+50) and projectile.alive==True: #je verifie que le projectile soit dans la zone de l'alien je n'ai pas utilisé find_overlapping puisque mes images ne renvoie que 2 coordonnées
                    self.alive =False
                    projectile.alive=False #desactivation du projectile et de l'alien de manière à les rendre invisible
                    projectile.move_in(canvas)
                    self.img =tk.PhotoImage(file='vidos.png')
                    self.id=canvas.create_image(x2,y2, image=self.img)
    def install_in(self, canvas,dx,dy):
        canvas.pack()
        self.id=canvas.create_image(dx,dy, image=self.img)
    def move_in(self, canvas):
        x1 = canvas.coords(self.id)[0]
        y1 = canvas.coords(self.id)[1]
        canvas.move(self.id,self.speed,0)

In [5]:
class Fleet(object):
    def __init__(self):
        self.aliens_lines = 3
        self.aliens_columns = 6
        self.aliens_inner_gap = 20
        self.fleet_size =self.aliens_lines * self.aliens_columns
        self.aliens_fleet = [None] * self.fleet_size
        self.x=35
        self.y=50
        self.compteur=2 #Le compteur permet de respecter le nombre de colonne mentionnée plus haut
        self.alien=Alien()
        self.speed=35
    def install_in(self,canvas):
            x,y = 40,20
            for i in range(len(self.aliens_fleet)):
                self.aliens_fleet[i]=Alien()
                self.aliens_fleet[i].install_in(canvas, x, y)
                x=x+60+self.aliens_inner_gap
                if self.compteur%self.aliens_columns==1: #le modulo permet de respecter le nombre de colonne
                    x=40
                    y=y+53+self.aliens_inner_gap
                self.compteur+=1
    def move_in(self, canvas):
        for i in range(0,18):
            canvas.move(self.aliens_fleet[i].id,0,self.speed) #le self.speed est à la position de la hauteur pour pouvoir gerer le deplacement verticale quand la flotte touche un mur
            
    def manage_touched_aliens_by(self,canvas,defender):
        for bullet in defender.fired_bullets.copy():
            for i in range(0,18):
                self.aliens_fleet[i].touched_by(canvas,bullet) #cette double boucle permet de recuperer les ID de chaque bullet et alien pour savoir si l'un à été touché par un autre

In [6]:
class Score():
    def __init__(self,pseudo,killedennemies,playtime):
        self.pseudo=pseudo
        self.killedennemies=killedennemies
        self.playtime=playtime
    def getpseudo(self):
        return self.pseudo
    def getkilledennemies(self):
        return self.killedennemies
    def getplaytime(self):
        return self.playtime
    def toFile(self, fich):
        f = open(fich,"w")
        l=self
        json.dump(l.__dict__,f)
        f.close()
    @classmethod
    def fromFile(cls, fich):
        f = open(fich,"r")
        d = json.load(f)
        snew=Score(d["pseudo"], d["killedennemies"],d["playtime"])
        f.close()
        return snew

In [7]:
class Game(object):
    def __init__(self,frame,root):
        self.frame = frame
        self.root = root
        self.defender = Defender()
        self.fleet=Fleet()
        self.height =800
        self.width =700
        self.canvas = tk.Canvas(self.frame, width=self.width,height=self.height,bg='black')
        self.canvas.pack()
        self.defender.install_in(self.canvas)
        self.fleet.install_in(self.canvas)
        self.compt=0#Permet de définir si on a gagné ou non
        self.compt_copy=0#Permet de garder une copie du score dans un coin protégé
        self.img=tk.PhotoImage(file='nuke.png') #préchargement de la photo de la victoire quand vous reussissez à éliminer chaque alien du niveau
        self.id=[None]
        self.tempsdedepart=datetime.now()
        self.scoretemporel=None
        self.pseudo="un joueur inconnu"
        self.old_score=Score.fromFile("score.json")
        self.canvas.create_text(350,750,fill='white',text="Dans la partie précédente,  " + self.old_score.getpseudo() +  " a tué " + str(self.old_score.getkilledennemies()) + " ennemis., en un temps de : " + self.old_score.getplaytime() + " secondes." )
    def animation(self):
        self.move_bullets() #Permet de déplacer les missiles tirés
        self.move_fleet() #Permet de déplacer les aliens
        self.fleet.manage_touched_aliens_by(self.canvas,self.defender) #Sert à savoir si un alien à été touché par un missile
        endgame=self.win() #Appelle la fonction relative à la victoire pour savoir si vous avez gagné ou perdu
        if endgame==0:
            self.canvas.after(250, self.animation)#Se rappelle par lui même pour pouvoir avoir des mouvements en continu
        elif endgame==1:
            self.score=Score(self.pseudo,self.compt_copy,str(self.scoretemporel))#Hop on garde en mémoire le score
            #self.pseudo=Entry(self.root,back) ça fonctionne pas , on a du mal a comprendre comment utiliser le entry, puisqu'il est pas dans le canvas
            self.score.toFile("score.json") #On enregistre le score dans le fichier score.json
    def start_animation(self):
        self.canvas.after(10, self.animation)
    def move_bullets(self):
        for bullet in self.defender.fired_bullets.copy():
            bullet.move_in(self.canvas)
            x1,y1=self.canvas.coords(bullet.id)
            if y1<0:
                self.canvas.delete(bullet.id) #Supprime le missile qui est sorti de la fenetre
                self.defender.fired_bullets.remove(bullet)#Permet de recharger les missiles du vaisseau
    def keypress(self,event): #Sert a savoir si le joueur appuye sur une touche ou nom
        if event.keysym == 'Left':
            self.defender.move_in(self.canvas,-15)
        if event.keysym == 'Right':
            self.defender.move_in(self.canvas,15)
        if event.keysym == 'space':
            self.defender.fire(self.canvas)
    def move_fleet(self):
        for i in range(0,18): #Cette boucle permet de prendre en compte les identifiants de chacuns des aliens formant la flotte
            x,y=self.canvas.coords(self.fleet.aliens_fleet[i].id)
            if x>self.width or x<0 : #Sert à savoir si la flotte à touché les bords de la fenetre
                for i in range(0,18):
                    self.fleet.aliens_fleet[i].speed=self.fleet.aliens_fleet[i].speed*-1 #Cette ligne inverse le sens de direction de tous les aliens pour éviter qu'ils se bloquent contre le mur
                self.fleet.move_in(self.canvas)
        for i in range(0,18): 
            self.fleet.aliens_fleet[i].move_in(self.canvas)
    def win(self):
        for i in range(0,18):
            if self.fleet.aliens_fleet[i].alive==False: #Si un alien est mort on a 1 point sur 18
                self.compt+=1
            if self.compt==18: #Une fois tous les aliens tués vous avez gagné
                print('gagné')
                tempsfinal=datetime.now()
                self.scoretemporel=tempsfinal-self.tempsdedepart
                print(self.scoretemporel)
                self.img=tk.PhotoImage(file='nuke.png') #Chargement de l'image finale de victoire
                self.id=self.canvas.create_image(350,400, image=self.img)
                self.compt_copy=self.compt
                return 1
        for i in range(0,18):
            y=self.canvas.coords(self.fleet.aliens_fleet[i].id)[1]
            if y >= 670 and self.fleet.aliens_fleet[i].alive==True: #Si 1 alien vivant passe derrière le vaisseau vous avez perdu
                print('Perdu')
                tempsfinal=datetime.now()
                self.scoretemporel=tempsfinal-self.tempsdedepart
                print(self.scoretemporel)
                self.img=tk.PhotoImage(file='loose.png') #Chargement de l'image finale de defaite
                self.id=self.canvas.create_image(350,400, image=self.img)
                self.compt_copy=self.compt
                return 1
        self.compt=0
        return 0
    def endgame(self):
        pass
        

In [ ]:
class SpaceInvaders(object):
    def __init__(self):        
        self.root = tk.Tk()
        self.root.title("Space Invaders")
        self.frame = tk.Frame(self.root)
        self.frame.pack(side="top", fill="both")
        self.game = Game(self.frame,self.root)
    def play(self):
        self.root.bind("<Key>",self.game.keypress)
        self.game.start_animation()
        self.root.mainloop()
SpaceInvaders().play() #Lance le jeu